In [1]:
# Core
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline, RectBivariateSpline

# Astronomy
import astropy
from astropy.units import au, minute

from typing import List, Optional

In [2]:
# Local imports
import kepler_sieve
from asteroid_data import load_ast_pos
from asteroid_spline import get_df_shape, make_spline_df, spline_ast_data
# from asteroid_spline import make_spline_df, spline_ast_vec, spline_ast_elt
from planets_interp import get_earth_pos
from asteroid_direction import calc_dir_ast2obs, c
from asteroid_direction_test import jpl_ast_dir_populate
from ra_dec import radec2dir, astrometric_dir
from db_utils import sp2df, df2db
import matplotlib.pyplot as plt

In [3]:
# Build table JPL.AsteroidDirections from JPl.AsteroidDirections_Import
jpl_ast_dir_populate()

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
# Inputs
n0 = 1
n1 = 3

In [11]:
# Import the JPL asteroid directions
sp_name = 'JPL.GetAsteroidDirection'
params = {
    'n0': n0,
    'n1': n1,
}
dfh = sp2df(sp_name=sp_name, params=params)

In [12]:
# The Horizons DataFrame
dfh

,AsteroidID,ObservatoryID,TimeID,mjd,RA_ast,DEC_ast,ux_ast,uy_ast,uz_ast,RA_app,DEC_app,ux_app,uy_app,uz_app,LightTime
0,1,0,58176000,40400.0,315.984929,-26.691893,0.642520,-0.748252,-0.165186,315.540377,-26.813296,0.637003,-0.752949,-0.165212,17.119051
1,1,0,58183200,40405.0,315.339431,-27.239939,0.632400,-0.755451,-0.171357,314.893582,-27.359919,0.626840,-0.760065,-0.171385,16.848578
2,1,0,58190400,40410.0,314.556310,-27.799476,0.620634,-0.763823,-0.177167,314.108587,-27.917822,0.615018,-0.768345,-0.177196,16.625932
3,1,0,58197600,40415.0,313.649961,-28.358533,0.607416,-0.773133,-0.182513,313.200319,-28.474913,0.601738,-0.777553,-0.182544,16.454580
4,1,0,58204800,40420.0,312.640884,-28.903941,0.593018,-0.783102,-0.187298,312.189150,-29.018219,0.587274,-0.787411,-0.187332,16.337417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29759,2,1,111715200,77580.0,305.864105,11.497201,0.574109,-0.649331,0.498766,306.711224,11.730989,0.585296,-0.639270,0.498760,29.139334
29760,2,1,111722400,77585.0,306.730950,12.128593,0.584709,-0.635330,0.504452,307.575687,12.367257,0.595658,-0.625081,0.504446,28.670323
29761,2,1,111729600,77590.0,307.512122,12.761502,0.593888,-0.621926,0.510398,308.354515,13.004506,0.604611,-0.611513,0.510390,28.191927
29762,2,1,111736800,77595.0,308.202299,13.391875,0.601624,-0.609253,0.516584,309.042243,13.638684,0.612132,-0.598703,0.516575,27.706250


In [13]:
# Calculate the direction and light time
df = calc_dir_ast2obs(n0=n0, n1=n1)
# Mask down to Ceres
mask = df.AsteroidID==1
dfm = df[mask]
# Take every 5th row to get data every 20 days
dfm = dfm.iloc[::5]
dfm.reset_index(inplace=True)

In [14]:
# MSE data frame for asteroid directions
dfm

,index,AsteroidID,TimeID,tAst,qAst_x,qAst_y,qAst_z,tObs,qObs_x,qObs_y,qObs_z,LightTime,ux,uy,uz
0,0,1,69120000,47999.983776,-1.155142,2.297488,0.282872,48000.0,-0.880033,-0.483797,-0.000050,23.362923,-0.097934,0.990084,0.100715
1,5,1,69148800,48019.982530,-1.339957,2.180571,0.313492,48020.0,-0.672050,-0.752861,-0.000054,25.156518,-0.220867,0.969778,0.103669
2,10,1,69177600,48039.981466,-1.515679,2.048533,0.341973,48040.0,-0.387430,-0.936156,-0.000062,26.689240,-0.351634,0.930049,0.106593
3,15,1,69206400,48059.980612,-1.680852,1.902253,0.368072,48060.0,-0.059128,-1.013903,-0.000068,27.919357,-0.483140,0.868647,0.109672
4,20,1,69235200,48079.979986,-1.834242,1.742653,0.391590,48080.0,0.275907,-0.978170,-0.000064,28.820858,-0.608968,0.785100,0.113026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746,3730,1,90604800,62919.983586,1.040245,2.523575,-0.111216,62920.0,-0.888358,0.439174,0.000041,23.635688,0.678583,0.733480,-0.039139
747,3735,1,90633600,62939.982228,0.841699,2.580912,-0.072849,62940.0,-0.989708,0.109224,0.000069,25.591547,0.595127,0.803283,-0.023686
748,3740,1,90662400,62959.981056,0.638171,2.622996,-0.034051,62960.0,-0.973270,-0.233765,0.000098,27.279676,0.491230,0.870968,-0.010400
749,3745,1,90691200,62979.980116,0.430794,2.649302,0.004953,62980.0,-0.842919,-0.549270,0.000119,28.633330,0.369905,0.929069,0.001415


In [15]:
# Get test DataFrame from JPL
sp_name = 'JPL.AsteroidDirectionTest'
dfh = sp2df(sp_name=sp_name)

# Take every 4th row to get data every 20 days; this matches exactly on MSE integration points
dfh = dfh.iloc[::4]
dfh.reset_index(inplace=True)

In [16]:
dfh

,index,AsteroidID,TimeID,tAst,qObs_x,qObs_y,qObs_z,qAst_x,qAst_y,qAst_z,...,vAst_y,vAst_z,LightTime,tObs,ux_ast,uy_ast,uz_ast,ux_app,uy_app,uz_app
0,0,1,69120000,47999.983775,-0.880028,-0.483807,-0.000050,-1.155150,2.297484,0.282873,...,-0.005464,0.001583,23.363621,48000.0,-0.097885,0.990090,0.100704,-0.095560,0.990312,0.100744
1,4,1,69134400,48009.983132,-0.787422,-0.627462,-0.000056,-1.248709,2.240893,0.298453,...,-0.005853,0.001533,24.289747,48010.0,-0.157895,0.982153,0.102200,-0.155585,0.982517,0.102239
2,8,1,69148800,48019.982530,-0.672042,-0.752868,-0.000054,-1.340122,2.180457,0.313519,...,-0.006233,0.001480,25.156619,48020.0,-0.220819,0.969791,0.103659,-0.218535,0.970304,0.103696
3,12,1,69163200,48029.981973,-0.537449,-0.856722,-0.000062,-1.429222,2.116260,0.328044,...,-0.006605,0.001424,25.958353,48030.0,-0.285668,0.952547,0.105113,-0.283425,0.953212,0.105149
4,16,1,69177600,48039.981466,-0.387421,-0.936160,-0.000062,-1.515844,2.048398,0.342000,...,-0.006966,0.001366,26.689323,48040.0,-0.351587,0.930068,0.106584,-0.349400,0.930888,0.106617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24003,96012,16,90655200,62954.983042,-0.988478,-0.149078,0.000090,-0.554700,2.752636,-0.114071,...,-0.000665,0.000315,24.419855,62955.0,0.147790,0.988254,-0.038882,0.140214,0.989357,-0.038904
24004,96016,16,90669600,62964.982230,-0.950906,-0.316688,0.000103,-0.658711,2.744160,-0.110849,...,-0.001029,0.000330,25.589057,62965.0,0.095022,0.994822,-0.036062,0.087406,0.995519,-0.036085
24005,96020,16,90684000,62974.981453,-0.885496,-0.475018,0.000111,-0.761859,2.732075,-0.107481,...,-0.001387,0.000344,26.707754,62975.0,0.038555,0.998695,-0.033506,0.030914,0.998960,-0.033529
24006,96024,16,90698400,62984.980719,-0.794209,-0.619483,0.000126,-0.864021,2.716450,-0.103974,...,-0.001737,0.000357,27.764006,62985.0,-0.020859,0.999296,-0.031185,-0.028499,0.999106,-0.031210


In [ ]:
# Relevant columns for astrom